In [1]:
import os, sys, glob, random
import numpy as np
import pandas as pd
import SimpleITK as sitk
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, roc_auc_score, average_precision_score, confusion_matrix, brier_score_loss
from hyperopt import hp, tpe, Trials
from hyperopt.fmin import fmin
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

import compare_auc_delong_xu

np.random.seed(529)
scaler = StandardScaler()

In [2]:
def getstats(y_true,y_pred,y_prob):
    rocauc = roc_auc_score(y_true,y_prob[:,1])
    brsc = round(brier_score_loss(y_true, y_prob[:,1]),2)
    prauc = round(average_precision_score(y_true,y_prob[:,1]),2)
    TN, FP, FN, TP = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    pos_pred_val = round(TP/ (TP+FP),2)
    neg_pred_val = round(TN/ (TN+FN),2)
    stats = [rocauc, prauc, brsc, pos_pred_val, neg_pred_val]
    return stats

def getAUC_CI(labels,scores):
    alpha = 0.95
    aucs = roc_auc_score(labels,scores)
    auc_delong, variances = compare_auc_delong_xu.delong_roc_variance(labels,scores)
    auc_std = np.sqrt(variances)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)
    ci = stats.norm.ppf(lower_upper_q,loc=auc_delong,scale=auc_std)
    return auc_delong, variances, ci

In [3]:
basefile = pd.read_csv('/data/project/bhattlab/boddugit/datafolder/COPD_P1_P2_P3_10.2021.csv',low_memory=False)
basefile = basefile[['sid','FEV1_FVC_post_P3']]
basefile['COPD_P3'] = pd.cut(basefile['FEV1_FVC_post_P3'], bins=[0, 0.7, float('Inf')], labels=[1, 0])
basefile = basefile[['sid','COPD_P3']]
basefile = basefile.dropna()

# merging the base file with radiomics feature file
df1 = pd.read_csv('/data/project/bhattlab/appr/PyRadiomics_features_set/Phase3/phase3_data.csv')
df1base = pd.merge(df1,basefile,on='sid')
print('Size after merging with radiomics feature file',df1base.shape)
print('Class Distribution\n',df1base['COPD_P3_x'].value_counts())

#saving the file for Table 1 and Table 3 measurements
df1.to_csv('low-dose-dataframe-radiology.csv',index=False)


X1 = df1base.iloc[:,1:124].astype('float64')
y1 = df1base['COPD_P3_x']

Size after merging with radiomics feature file (1688, 126)
Class Distribution
 0    943
1    745
Name: COPD_P3_x, dtype: int64


In [4]:
seed = 529
Xt, Xv, yt, yv = train_test_split(X1, y1, test_size = 0.2, random_state = seed, stratify=y1)
Xt, Xv = np.array(Xt), np.array(Xv)
print(Xt.shape,Xv.shape)
Xtdemo, Xtqct, Xtrad = Xt[:,0:6], Xt[:,6:7], Xt[:,8:]
Xvdemo, Xvqct, Xvrad = Xv[:,0:6], Xv[:,6:7], Xv[:,8:]
Xtrad, Xvrad = StandardScaler().fit_transform(Xtrad), StandardScaler().fit_transform(Xvrad)
Xtcomb = np.concatenate((Xtdemo,Xtqct,Xtrad),axis=1)
Xvcomb = np.concatenate((Xvdemo,Xvqct,Xvrad),axis=1)
print(Xtrad.shape,Xtcomb.shape)

clfdemo = CatBoostClassifier(loss_function='CrossEntropy')
clfdemo.fit(Xtdemo,yt)
y_pred_demo = clfdemo.predict(Xvdemo)
y_prob_demo = clfdemo.predict_proba(Xvdemo)
demostats = getstats(yv,y_pred_demo,y_prob_demo)

clfqct = CatBoostClassifier(loss_function='CrossEntropy')
clfqct.fit(Xtqct,yt)
y_pred_qct = clfqct.predict(Xvqct)
y_prob_qct = clfqct.predict_proba(Xvqct)
qctstats = getstats(yv,y_pred_qct,y_prob_qct)

clfrad = CatBoostClassifier(loss_function='CrossEntropy')
clfrad.fit(Xtrad,yt)
y_pred_rad = clfrad.predict(Xvrad)
y_prob_rad = clfrad.predict_proba(Xvrad)
radstats = getstats(yv,y_pred_rad,y_prob_rad)

clfcomb = CatBoostClassifier(loss_function='CrossEntropy')
clfcomb.fit(Xtcomb,yt)
y_pred_comb = clfcomb.predict(Xvcomb)
y_prob_comb = clfcomb.predict_proba(Xvcomb)
combstats = getstats(yv,y_pred_comb,y_prob_comb)

print()
print('rocauc', 'prauc', 'sensitivity', 'specificity', 'pos_pred_val', 'neg_pred_val')
print('demographics -> AUC, PRAUC, BRIER, PPV, NPV', demostats)
print('CT emphysema -> AUC, PRAUC, BRIER, PPV, NPV', qctstats)
print('CT Radiomics -> AUC, PRAUC, BRIER, PPV, NPV', combstats)
print('Combined     -> AUC, PRAUC, BRIER, PPV, NPV', combstats)

(1350, 123) (338, 123)
(1350, 115) (1350, 122)
0:	learn: 0.6873329	total: 52.1ms	remaining: 52s
1:	learn: 0.6816544	total: 54.2ms	remaining: 27.1s
2:	learn: 0.6755357	total: 56ms	remaining: 18.6s
3:	learn: 0.6706456	total: 57ms	remaining: 14.2s
4:	learn: 0.6658393	total: 57.8ms	remaining: 11.5s
5:	learn: 0.6603432	total: 59.4ms	remaining: 9.84s
6:	learn: 0.6559000	total: 61.3ms	remaining: 8.7s
7:	learn: 0.6516425	total: 62.8ms	remaining: 7.79s
8:	learn: 0.6471881	total: 63.8ms	remaining: 7.03s
9:	learn: 0.6435835	total: 65.7ms	remaining: 6.5s
10:	learn: 0.6396834	total: 66.5ms	remaining: 5.97s
11:	learn: 0.6357766	total: 67.2ms	remaining: 5.54s
12:	learn: 0.6318505	total: 68.1ms	remaining: 5.17s
13:	learn: 0.6280551	total: 68.9ms	remaining: 4.85s
14:	learn: 0.6246891	total: 69.7ms	remaining: 4.57s
15:	learn: 0.6217225	total: 70.8ms	remaining: 4.35s
16:	learn: 0.6196762	total: 71.3ms	remaining: 4.12s
17:	learn: 0.6163861	total: 72.8ms	remaining: 3.97s
18:	learn: 0.6136174	total: 73.6ms	

254:	learn: 0.4717286	total: 306ms	remaining: 894ms
255:	learn: 0.4714058	total: 307ms	remaining: 892ms
256:	learn: 0.4710793	total: 308ms	remaining: 890ms
257:	learn: 0.4706194	total: 309ms	remaining: 888ms
258:	learn: 0.4700751	total: 310ms	remaining: 887ms
259:	learn: 0.4696018	total: 311ms	remaining: 885ms
260:	learn: 0.4691294	total: 312ms	remaining: 883ms
261:	learn: 0.4686907	total: 313ms	remaining: 880ms
262:	learn: 0.4680101	total: 313ms	remaining: 878ms
263:	learn: 0.4677230	total: 314ms	remaining: 876ms
264:	learn: 0.4674492	total: 315ms	remaining: 874ms
265:	learn: 0.4673409	total: 317ms	remaining: 873ms
266:	learn: 0.4670328	total: 317ms	remaining: 871ms
267:	learn: 0.4660101	total: 318ms	remaining: 869ms
268:	learn: 0.4656613	total: 319ms	remaining: 867ms
269:	learn: 0.4651550	total: 320ms	remaining: 865ms
270:	learn: 0.4648580	total: 321ms	remaining: 864ms
271:	learn: 0.4644821	total: 322ms	remaining: 861ms
272:	learn: 0.4640363	total: 323ms	remaining: 859ms
273:	learn: 

452:	learn: 0.3937077	total: 497ms	remaining: 600ms
453:	learn: 0.3932627	total: 498ms	remaining: 598ms
454:	learn: 0.3927287	total: 498ms	remaining: 597ms
455:	learn: 0.3922745	total: 499ms	remaining: 595ms
456:	learn: 0.3920860	total: 500ms	remaining: 594ms
457:	learn: 0.3919051	total: 501ms	remaining: 593ms
458:	learn: 0.3916077	total: 502ms	remaining: 591ms
459:	learn: 0.3911065	total: 503ms	remaining: 590ms
460:	learn: 0.3904629	total: 504ms	remaining: 589ms
461:	learn: 0.3903984	total: 505ms	remaining: 588ms
462:	learn: 0.3898826	total: 506ms	remaining: 586ms
463:	learn: 0.3893301	total: 506ms	remaining: 585ms
464:	learn: 0.3888377	total: 508ms	remaining: 584ms
465:	learn: 0.3886403	total: 509ms	remaining: 583ms
466:	learn: 0.3885270	total: 510ms	remaining: 582ms
467:	learn: 0.3878854	total: 511ms	remaining: 581ms
468:	learn: 0.3874863	total: 511ms	remaining: 579ms
469:	learn: 0.3870737	total: 512ms	remaining: 578ms
470:	learn: 0.3869418	total: 513ms	remaining: 576ms
471:	learn: 

657:	learn: 0.3360324	total: 692ms	remaining: 360ms
658:	learn: 0.3357171	total: 693ms	remaining: 358ms
659:	learn: 0.3356230	total: 694ms	remaining: 357ms
660:	learn: 0.3355082	total: 695ms	remaining: 356ms
661:	learn: 0.3353985	total: 696ms	remaining: 355ms
662:	learn: 0.3351049	total: 697ms	remaining: 354ms
663:	learn: 0.3347025	total: 698ms	remaining: 353ms
664:	learn: 0.3343060	total: 698ms	remaining: 352ms
665:	learn: 0.3339413	total: 699ms	remaining: 351ms
666:	learn: 0.3333830	total: 700ms	remaining: 350ms
667:	learn: 0.3330073	total: 701ms	remaining: 349ms
668:	learn: 0.3327869	total: 702ms	remaining: 347ms
669:	learn: 0.3325342	total: 703ms	remaining: 346ms
670:	learn: 0.3322402	total: 705ms	remaining: 346ms
671:	learn: 0.3321915	total: 706ms	remaining: 344ms
672:	learn: 0.3320795	total: 707ms	remaining: 343ms
673:	learn: 0.3319130	total: 708ms	remaining: 342ms
674:	learn: 0.3315526	total: 709ms	remaining: 341ms
675:	learn: 0.3312617	total: 710ms	remaining: 340ms
676:	learn: 

819:	learn: 0.2973435	total: 887ms	remaining: 195ms
820:	learn: 0.2972407	total: 888ms	remaining: 194ms
821:	learn: 0.2968796	total: 888ms	remaining: 192ms
822:	learn: 0.2967463	total: 890ms	remaining: 191ms
823:	learn: 0.2965950	total: 891ms	remaining: 190ms
824:	learn: 0.2964457	total: 892ms	remaining: 189ms
825:	learn: 0.2961367	total: 893ms	remaining: 188ms
826:	learn: 0.2960243	total: 894ms	remaining: 187ms
827:	learn: 0.2957142	total: 895ms	remaining: 186ms
828:	learn: 0.2954799	total: 896ms	remaining: 185ms
829:	learn: 0.2953015	total: 897ms	remaining: 184ms
830:	learn: 0.2950169	total: 897ms	remaining: 183ms
831:	learn: 0.2950079	total: 898ms	remaining: 181ms
832:	learn: 0.2948031	total: 899ms	remaining: 180ms
833:	learn: 0.2945367	total: 900ms	remaining: 179ms
834:	learn: 0.2943845	total: 901ms	remaining: 178ms
835:	learn: 0.2942338	total: 901ms	remaining: 177ms
836:	learn: 0.2942015	total: 902ms	remaining: 176ms
837:	learn: 0.2940711	total: 903ms	remaining: 175ms
838:	learn: 

...

306:	learn: 0.1709817	total: 2.5s	remaining: 5.66s
307:	learn: 0.1705070	total: 2.51s	remaining: 5.64s
308:	learn: 0.1701836	total: 2.52s	remaining: 5.63s
309:	learn: 0.1697218	total: 2.53s	remaining: 5.62s
310:	learn: 0.1692127	total: 2.53s	remaining: 5.62s
311:	learn: 0.1687764	total: 2.54s	remaining: 5.6s
312:	learn: 0.1683042	total: 2.55s	remaining: 5.59s
313:	learn: 0.1679479	total: 2.56s	remaining: 5.58s
314:	learn: 0.1669246	total: 2.57s	remaining: 5.59s
315:	learn: 0.1664017	total: 2.58s	remaining: 5.59s
316:	learn: 0.1658037	total: 2.59s	remaining: 5.58s
317:	learn: 0.1651850	total: 2.6s	remaining: 5.57s
318:	learn: 0.1644362	total: 2.6s	remaining: 5.56s
319:	learn: 0.1639202	total: 2.61s	remaining: 5.55s
320:	learn: 0.1635374	total: 2.62s	remaining: 5.54s
321:	learn: 0.1630016	total: 2.63s	remaining: 5.53s
322:	learn: 0.1624493	total: 2.63s	remaining: 5.52s
323:	learn: 0.1620437	total: 2.64s	remaining: 5.51s
324:	learn: 0.1615875	total: 2.65s	remaining: 5.5s
325:	learn: 0.160

471:	learn: 0.0990913	total: 3.95s	remaining: 4.42s
472:	learn: 0.0988048	total: 3.96s	remaining: 4.41s
473:	learn: 0.0983499	total: 3.97s	remaining: 4.4s
474:	learn: 0.0980956	total: 3.98s	remaining: 4.39s
475:	learn: 0.0977788	total: 3.98s	remaining: 4.38s
476:	learn: 0.0973535	total: 3.99s	remaining: 4.37s
477:	learn: 0.0969365	total: 4s	remaining: 4.37s
478:	learn: 0.0965558	total: 4s	remaining: 4.36s
479:	learn: 0.0961880	total: 4.01s	remaining: 4.35s
480:	learn: 0.0958655	total: 4.02s	remaining: 4.34s
481:	learn: 0.0956253	total: 4.03s	remaining: 4.33s
482:	learn: 0.0951418	total: 4.03s	remaining: 4.32s
483:	learn: 0.0949384	total: 4.04s	remaining: 4.31s
484:	learn: 0.0947090	total: 4.05s	remaining: 4.3s
485:	learn: 0.0944494	total: 4.06s	remaining: 4.29s
486:	learn: 0.0941845	total: 4.06s	remaining: 4.28s
487:	learn: 0.0938973	total: 4.07s	remaining: 4.27s
488:	learn: 0.0937070	total: 4.08s	remaining: 4.26s
489:	learn: 0.0932228	total: 4.08s	remaining: 4.25s
490:	learn: 0.092942

657:	learn: 0.0584893	total: 5.39s	remaining: 2.8s
658:	learn: 0.0583715	total: 5.4s	remaining: 2.79s
659:	learn: 0.0582036	total: 5.41s	remaining: 2.79s
660:	learn: 0.0579987	total: 5.45s	remaining: 2.79s
661:	learn: 0.0579043	total: 5.46s	remaining: 2.79s
662:	learn: 0.0576811	total: 5.46s	remaining: 2.78s
663:	learn: 0.0574534	total: 5.47s	remaining: 2.77s
664:	learn: 0.0572854	total: 5.48s	remaining: 2.76s
665:	learn: 0.0571924	total: 5.49s	remaining: 2.75s
666:	learn: 0.0569570	total: 5.5s	remaining: 2.74s
667:	learn: 0.0568483	total: 5.5s	remaining: 2.73s
668:	learn: 0.0567166	total: 5.51s	remaining: 2.73s
669:	learn: 0.0565704	total: 5.52s	remaining: 2.72s
670:	learn: 0.0563523	total: 5.53s	remaining: 2.71s
671:	learn: 0.0563354	total: 5.53s	remaining: 2.7s
672:	learn: 0.0562468	total: 5.54s	remaining: 2.69s
673:	learn: 0.0560696	total: 5.55s	remaining: 2.68s
674:	learn: 0.0559622	total: 5.56s	remaining: 2.67s
675:	learn: 0.0558183	total: 5.56s	remaining: 2.67s
676:	learn: 0.055

836:	learn: 0.0380184	total: 6.85s	remaining: 1.33s
837:	learn: 0.0379603	total: 6.86s	remaining: 1.33s
838:	learn: 0.0379196	total: 6.88s	remaining: 1.32s
839:	learn: 0.0378257	total: 6.89s	remaining: 1.31s
840:	learn: 0.0377585	total: 6.9s	remaining: 1.3s
841:	learn: 0.0376954	total: 6.91s	remaining: 1.29s
842:	learn: 0.0375898	total: 6.91s	remaining: 1.29s
843:	learn: 0.0374780	total: 6.92s	remaining: 1.28s
844:	learn: 0.0374033	total: 6.93s	remaining: 1.27s
845:	learn: 0.0373370	total: 6.94s	remaining: 1.26s
846:	learn: 0.0372904	total: 6.94s	remaining: 1.25s
847:	learn: 0.0372360	total: 6.95s	remaining: 1.25s
848:	learn: 0.0371733	total: 6.96s	remaining: 1.24s
849:	learn: 0.0370881	total: 6.97s	remaining: 1.23s
850:	learn: 0.0370240	total: 6.98s	remaining: 1.22s
851:	learn: 0.0369085	total: 6.99s	remaining: 1.21s
852:	learn: 0.0368176	total: 6.99s	remaining: 1.21s
853:	learn: 0.0367681	total: 7s	remaining: 1.2s
854:	learn: 0.0367293	total: 7.01s	remaining: 1.19s
855:	learn: 0.0366

997:	learn: 0.0271911	total: 8.15s	remaining: 16.3ms
998:	learn: 0.0271415	total: 8.16s	remaining: 8.17ms
999:	learn: 0.0270883	total: 8.17s	remaining: 0us

rocauc prauc sensitivity specificity pos_pred_val neg_pred_val
demographics -> AUC, PRAUC, BRIER, PPV, NPV [0.7000461631334115, 0.64, 0.23, 0.57, 0.66]
CT emphysema -> AUC, PRAUC, BRIER, PPV, NPV [0.7466531728276695, 0.72, 0.2, 0.63, 0.72]
CT Radiomics -> AUC, PRAUC, BRIER, PPV, NPV [0.8888178686836404, 0.88, 0.14, 0.79, 0.8]
Combined     -> AUC, PRAUC, BRIER, PPV, NPV [0.8888178686836404, 0.88, 0.14, 0.79, 0.8]


In [7]:
aucdemo, aucvardemo, auccidemo = getAUC_CI(yv,y_prob_demo[:,1])
aucqct, aucvarqct, aucciqct = getAUC_CI(yv,y_prob_qct[:,1])
aucrad, aucvarrad, auccirad = getAUC_CI(yv,y_prob_rad[:,1])
auccomb, aucvarcomb, auccicomb = getAUC_CI(yv,y_prob_comb[:,1])

print('demographics -> CI ',aucdemo,auccidemo)
print('CT emphysema -> CI ',aucqct,aucciqct)
print('CT radiomics -> CI ',aucrad, auccirad)
print('combined     => CI ',auccomb, auccicomb)

auc_rad_vs_demo = compare_auc_delong_xu.delong_roc_test(yv,y_prob_rad[:,1],y_prob_demo[:,1])
auc_rad_vs_qct = compare_auc_delong_xu.delong_roc_test(yv,y_prob_rad[:,1],y_prob_qct[:,1])
auc_rad_vs_comb = compare_auc_delong_xu.delong_roc_test(yv,y_prob_rad[:,1],y_prob_comb[:,1])
print('AUC Comparison - Demographics vs. Radiomics, DeLong P-value ', np.exp(auc_rad_vs_demo))
print('AUC Comparison - CT emphysema vs. Radiomics, DeLong P-value ',np.exp(auc_rad_vs_qct))
print('AUC Comparison - Combined vs. Radiomics, DeLong P-value ',np.exp(auc_rad_vs_comb))


lowdose_feature_imp = pd.DataFrame(sorted(zip(clfrad.feature_importances_,X1.columns[8:])), columns=['Value','Feature'])
lowdose_feature_imp = lowdose_feature_imp.sort_values(by="Value", ascending=False)
lowdose_feature_imp[0:10]

demographics -> CI  0.7000461631334116 [0.64498966 0.75510266]
CT emphysema -> CI  0.7466531728276695 [0.69485187 0.79845448]
CT radiomics -> CI  0.8718085295266504 [0.83324468 0.91037238]
combined     => CI  0.8888178686836405 [0.85465077 0.92298497]
AUC Comparison - Demographics vs. Radiomics, DeLong P-value  [[0.00115041]]
AUC Comparison - CT emphysema vs. Radiomics, DeLong P-value  [[0.002989]]
AUC Comparison - Combined vs. Radiomics, DeLong P-value  [[0.32430733]]


,Value,Feature
114,5.517764,original_shape_Sphericity
113,4.205237,rightoriginal_shape_Sphericity
112,3.229163,original_glszm_GrayLevelNonUniformityNormalized
111,2.846843,original_firstorder_10Percentile
110,2.494523,leftoriginal_shape_Sphericity
109,2.368065,original_glrlm_LongRunLowGrayLevelEmphasis
108,2.304361,original_shape_SurfaceVolumeRatio
107,2.294795,original_glrlm_RunEntropy
106,1.995239,rightoriginal_shape_Flatness
105,1.820820,original_shape_LeastAxisLength
